RAG - Loader(UpstageDocumentParseLoader)

In [ ]:
# .env 파일 로드
from dotenv import load_dotenv
load_dotenv()  # .env 파일에서 환경변수 로드

#로더 실행
from langchain_upstage import UpstageDocumentParseLoader

file_path = "./test_modified.pdf"

loader = UpstageDocumentParseLoader(
    file_path,
    split="page",  # 페이지별로 분할
    output_format="text",  # 텍스트 형태로 출력
    ocr="auto",  # 자동 OCR 사용
    coordinates=True,  # 좌표 정보 포함
)


docs = loader.load()

In [ ]:
print(docs[21].page_content)

Rag - Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#  문서 자르기 (AI가 읽기 쉬운 크기로 쪼개기)
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
docs_spliiter = splitter.split_documents(docs)

In [ ]:
len(docs_spliiter)

52

Rag - Embedding

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings


model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings = HuggingFaceEmbeddings(model="intfloat/multilingual-e5-large-instruct")

dimension_size = len(embeddings.embed_query("hello world"))
print(dimension_size)

1024


Rag - Vector Store

In [ ]:
from langchain_community.vectorstores import FAISS
import faiss

db = FAISS(
    embedding_function=embeddings,
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# split_doc1 → docs 또는 올바른 변수명 사용
db = FAISS.from_documents(documents=docs, embedding=hf_embeddings)

In [ ]:
db.index_to_docstore_id

In [ ]:
db.docstore._dict

Rag - retriever

In [ ]:
# 검색 테스트
query = "아주대 기계공학과 고교추천전형 몇명 모집해??"
results = db.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n=== {i+1}번째 결과 ===")
    print(f"내용: {doc.page_content[:700]}...")
    print(f"페이지: {doc.metadata.get('page', 'N/A')}")

In [ ]:
vectorstore = FAISS.from_documents(docs, hf_embeddings)

In [70]:
retriever = vectorstore.as_retriever()

Rag - Formating / Prompt

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [ ]:
load_dotenv()


llm = ChatAnthropic(
    model="claude-3-5-haiku-20241022", temperature=0  # 최신 Claude3.5 Sonnet 예시
)


# 📝 문서 포맷팅 함수 (중요!)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


prompt = ChatPromptTemplate.from_template(
    """
아래 문서를 참고하여 질문에 답하세요.
모르겠으면 모른다고 솔직하게 답하세요.

문서:
{context}

질문: {question}
"""
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Rag - QnA

In [100]:
# 질문하기 - 파이프 방식
question = "아주대 산업공학 수시로 몇명 뽑아 ?"
response = rag_chain.invoke(question)  # 직접 문자열 전달 (딕셔너리가 아님!)
print("💬 AI의 답변:\n")
print(response)  # StrOutputParser() 덕분에 바로 문자열로 출력됨

💬 AI의 답변:

문서를 보면 산업공학과의 수시모집 인원은 총 20명입니다.
